In [87]:
import pandas as pd
import numpy as np

In [9]:
# percent cover data
df_pc = pd.read_table('relevant/percent_cover_zipcode.txt', delimiter = ',')
df_pc.columns = ['zip', 'pc_cover']
df_pc_nyc = df_pc.loc[(df_pc['zip'] >= 10001) & (df_pc['zip'] <= 11697)]

In [21]:
df_cities_zips = pd.read_csv('relevant/us_cities_zips.csv')
# nyc zip codes: 10001 to 11697
df_zip_nyc = df_cities_zips.loc[(df_cities_zips['zip_code'] >= 10001) & (df_cities_zips['zip_code'] <= 11697)]
df_zip_nyc_relevant = df_zip_nyc[['zip_code', 'population', 'density']]
df_zip_nyc_relevant.columns = ['zip', 'population', 'density']

In [30]:
# merge datasets
combined = pd.merge(df_zip_nyc_relevant, df_pc_nyc, how="outer")
combined = combined.dropna()

In [95]:
# covid data
df_hosprate = pd.read_csv('relevant/covid/hosprate-by-modzcta.csv')
df_deathrate = pd.read_csv('relevant/covid/deathrate-by-modzcta.csv')
df_percentpos = pd.read_csv('relevant/covid/percentpositive-by-modzcta.csv')

# df_hosprate = df_hosprate.transpose()
# df_hosprate.columns = df_hosprate.iloc[0]
# df_hosprate.drop(df_hosprate.index[0])

In [123]:
# get hosp rate averages by zip code
temp = []
cols = df_hosprate.columns
for i in range(7, len(cols)):
    hosp_avg = np.nanmean(list(df_hosprate[df_hosprate.columns[i]]))
    death_avg = np.nanmean(list(df_deathrate[df_deathrate.columns[i]]))
    percentpos_avg = np.nanmean(list(df_percentpos[df_percentpos.columns[i]]))
    temp.append([cols[i][-5:], hosp_avg, death_avg, percentpos_avg])

In [124]:
covid_data = pd.DataFrame(data = temp, columns = ['zip', 'hosp_avg', 'death_avg', 'percentpos_avg'])
covid_data['zip']=covid_data['zip'].astype(int)

In [127]:
combined_all = combined.merge(covid_data, left_on='zip', right_on='zip')

In [49]:
df_hosprate['hosp_avg'] = df_hosprate.mean(axis=1)


In [56]:
df_hosprate.mean(numeric_only=True, axis = 0)

Series([], dtype: float64)

In [62]:
df_hosprate

date,3/2020,4/2020,5/2020,6/2020,7/2020,8/2020,9/2020,10/2020,11/2020,12/2020,...,4/2022,5/2022,6/2022,7/2022,8/2022,9/2022,10/2022,11/2022,12/2022,1/2023
date,3/2020,4/2020,5/2020,6/2020,7/2020,8/2020,9/2020,10/2020,11/2020,12/2020,...,4/2022,5/2022,6/2022,7/2022,8/2022,9/2022,10/2022,11/2022,12/2022,1/2023
HOSPRATE_Bronx,267.4,437.4,66.5,22.3,18.5,17.6,12.2,16.6,42.4,103.8,...,19.6,40.4,47.6,76.6,68.8,45.3,50.1,67.2,93.3,3.3
HOSPRATE_Brooklyn,207.1,300.9,45.5,21.4,14.0,12.0,14.8,25.8,36.1,91.6,...,18.0,34.9,40.5,55.4,49.4,38.7,41.0,54.3,66.9,1.9
HOSPRATE_Manhattan,167.0,235.7,33.3,12.5,10.7,7.1,6.1,10.2,25.8,66.7,...,22.2,38.0,37.7,56.3,44.3,31.5,40.3,51.3,67.4,2.1
HOSPRATE_Queens,266.0,394.8,47.6,18.3,13.5,10.8,10.4,17.6,40.1,96.5,...,17.5,34.3,37.8,57.9,42.7,39.1,44.4,54.3,72.0,2.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HOSPRATE_11691,342.5,767.3,98.7,31.4,12.0,19.4,6.0,28.4,35.9,94.2,...,13.5,56.8,28.4,56.8,46.4,20.9,28.4,68.8,82.3,4.5
HOSPRATE_11692,337.6,565.9,90.4,23.8,4.8,14.3,4.8,9.5,19.0,47.6,...,4.8,28.5,52.3,57.1,28.5,28.5,38.0,57.1,52.3,4.8
HOSPRATE_11693,284.1,244.6,23.7,7.9,NaN,NaN,15.8,NaN,31.6,63.1,...,NaN,23.7,39.5,23.7,31.6,63.1,31.6,55.2,39.5,NaN
HOSPRATE_11694,192.5,601.6,57.7,9.6,14.4,NaN,NaN,NaN,9.6,62.6,...,19.2,33.7,43.3,77.0,52.9,43.3,33.7,48.1,91.4,4.8
